In [1]:
import numpy as np
from msmbuilder.feature_selection import VarianceThreshold
from msmbuilder.decomposition import tICA
from msmbuilder.cluster import MiniBatchKMeans, KMeans, LandmarkAgglomerative, MiniBatchKMedoids, KMedoids
from msmbuilder.msm import MarkovStateModel
from sklearn.pipeline import Pipeline
from glob import glob

/home/robert/anaconda3/envs/science/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/robert/anaconda3/envs/science/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
# variance_cut
# 		 threshold 	 0.0
# tica
# 		 commute_mapping 	 False
# 		 kinetic_mapping 	 True
# 		 lag_time 	 1
# 		 n_components 	 4
# 		 shrinkage 	 None
# cluster
# 		 batch_size 	 100
# 		 compute_labels 	 True
# 		 init 	 k-means++
# 		 init_size 	 None
# 		 max_iter 	 100
# 		 max_no_improvement 	 10
# 		 n_clusters 	 566
# 		 n_init 	 3
# 		 random_state 	 None
# 		 reassignment_ratio 	 0.01
# 		 tol 	 0.0
# 		 verbose 	 0
# msm
# 		 ergodic_cutoff 	 on
# 		 lag_time 	 50
# 		 n_timescales 	 2
# 		 prior_counts 	 0
# 		 reversible_type 	 mle
# 		 sliding_window 	 True
# 		 use_gap 	 None
# 		 verbose 	 True

In [2]:
traj_paths = glob('/home/robert/Datasets/DHFR/train/pp_tor/trajectory-*.npy')

len(traj_paths)

350

In [3]:
trajs = [np.load(x) for x in traj_paths]

In [4]:
selector = VarianceThreshold()
vtrajs = selector.fit_transform(trajs)

In [5]:
len(selector.get_support(indices=True))

632

In [6]:
from sklearn.model_selection import ShuffleSplit

In [7]:
cv = ShuffleSplit(n_splits = 5, test_size=0.5, random_state=1)
for tr_idx, te_idx in cv.split(trajs):
    train = np.array(trajs)[tr_idx]
    selector = VarianceThreshold()
    selector.fit(train)
    print(len(selector.get_support(indices=True)))
    test = np.array(trajs)[te_idx]
    selector = VarianceThreshold()
    selector.fit(test)
    print(len(selector.get_support(indices=True)))
    print()

632
632

632
632

632
632

632
632

632
632



In [8]:
train = [np.array(trajs)[i] for i, _ in cv.split(trajs)]
test = [np.array(trajs)[i] for _, i in cv.split(trajs)]

In [9]:
train_idx = [i for i, _ in cv.split(trajs)]
test_idx = [i for _, i in cv.split(trajs)]

In [10]:
tica=tICA(kinetic_mapping=True, lag_time=76, n_components=9)
# cluster = MiniBatchKMeans(n_clusters=777, batch_size=10000)
cluster = KMeans(n_clusters=777, n_init=5)
# cluster = LandmarkAgglomerative(n_clusters=777, linkage='ward', n_landmarks=100)
msm = MarkovStateModel(lag_time=50, n_timescales=2)

In [11]:
ttrajs = tica.fit_transform(train[0])

In [12]:
ctrajs = cluster.fit_transform(ttrajs)

In [13]:
for ctraj in ctrajs:
    
    print(np.unique(ctraj))

[  3  47  54 113 140 174 216 227 229 236 248 252 264 269 298 346 354 374
 391 435 517 526 534 583 587 599 600 604 616 675 725]
[  9  37  69  96 121 149 182 202 211 257 380 443 478 497 560 627 672 674
 731 762 764]
[ 31 106 139 152 163 199 201 213 232 242 255 279 321 350 394 415 420 432
 486 518 528 597 601 610 635 638 645 670 682 686 714 728 748 750 758 759]
[ 69  96 121 202 211 257 443 478 497 560 627 672 674 764]
[  3  47  54  81 113 140 150 174 180 236 252 264 269 298 301 318 346 354
 374 391 435 517 554 568 583 587 599 600 604 658 725 753]
[ 65 111 119 206 396 543 731 739]
[  9  37  88 149 182 206 312 380 392 497 576 627 631 672 730 762 764]
[ 31 106 139 152 163 199 201 213 232 242 255 279 321 350 394 415 420 432
 486 515 518 528 597 601 610 635 638 645 670 686 714 728 737 748 750 758
 759]
[ 15  90 172 246 361 406 433 522 546 578 751 760]
[ 15  60  90 172 239 361 406 433 474 522 545 546 578 656 684 696 751 760]
[  3  47  54  81 113 140 150 174 180 236 252 264 269 298 301 346 374 3

In [14]:
mtrajs = msm.fit_transform(ctrajs)

MSM contains 22 strongly connected components above weight=0.02. Component 1 selected, with population 29.714286%


In [15]:
discarded = [x for x in range(777) if x not in msm.mapping_.keys()]
kept = [x for x in range(777) if x  in msm.mapping_.keys()]

In [16]:
print(len(kept), len(discarded), len(kept)+len(discarded))

186 591 777


In [17]:
cluster.landmark_labels_

AttributeError: 'KMeans' object has no attribute 'landmark_labels_'

In [18]:
# np.array([item in b for item in a])
discarded_frames = [np.where([item in discarded for item in cluster.labels_[i]])[0] for i in range(len(train[0]))] 
kept_frames = [np.where([item in kept for item in cluster.labels_[i] ])[0] for i in range(len(train[0]))] 

In [57]:
for i in range(175):
#     if len(discarded_frames[i])>0:
#         print('discarded: ', traj_paths[train_idx[0][i]])
    if len(kept_frames[i])>0:
        print('kept: ', traj_paths[train_idx[0][i]])
    print('-'*10)

----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-287.npy
----------
----------
----------
----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-233.npy
----------
----------
----------
----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-189.npy
----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-299.npy
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-379.npy
----------
----------
----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-281.npy
----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-209.npy
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-367.npy
----------
----------
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-293.npy
----------
kept:  /home/robert/Datasets/DHFR/train/pp_tor/trajectory-201.npy
----------
kept:  /home/robert/Datasets/DHFR/train/pp_